In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_url= "http://www-stat.wharton.upenn.edu/~waterman/DataSets/uva.txt"
df = pd.read_table(data_url)

In [ ]:
df.shape

In [ ]:
df[:5]

In [6]:
df_category_only = df.drop(['who', "Age", "Years on Internet"] , axis=1)
df_category_only[:5]

,Newbie,Gender,Household Income,Sexual Preference,Country,Education Attainment,Major Occupation,Marital Status
0,0,Male,$50-74,Gay male,Ontario,Some College,Computer,Other
1,0,Female,Over $100,Heterosexual,Sweden,Professional,Other,Other
2,1,Female,$40-49,Heterosexual,Washington,Some College,Management,Other
3,1,Female,$40-49,Heterosexual,Florida,Some College,Computer,Married
4,0,Male,$30-39,Bisexual,New Jersey,Some College,Education,Single


In [7]:
for col in ['Gender','Household Income', 'Sexual Preference', 'Education Attainment', 'Major Occupation', "Marital Status", "Country"]:
    df_category_only[col] = df_category_only[col].astype('category')
df_category_only.dtypes

Newbie                     int64
Gender                  category
Household Income        category
Sexual Preference       category
Country                 category
Education Attainment    category
Major Occupation        category
Marital Status          category
dtype: object

In [8]:
df_category_only = df_category_only.dropna()

In [9]:
df_dummies = pd.get_dummies(df_category_only)
df_dummies[:5]

,Newbie,Gender_Female,Gender_Male,Household Income_$10-19,Household Income_$20-29,Household Income_$30-39,Household Income_$40-49,Household Income_$50-74,Household Income_$75-99,Household Income_Over $100,...,Major Occupation_Education,Major Occupation_Management,Major Occupation_Other,Major Occupation_Professional,Marital Status_Divorced,Marital Status_Married,Marital Status_Other,Marital Status_Separated,Marital Status_Single,Marital Status_Widowed
0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
df_dummies.shape

(15553, 179)

In [106]:
x_data = df_dummies.ix[:, 1:].values
y_data = df_dummies.ix[:, 0].reshape(-1,1)

y_data.shape, x_data.shape

((15553, 1), (15553, 178))

In [229]:
def train_navie_bayes_possiblities(x_data, y_data):
    num_total_case = len(x_data)
    num_attributes = len(x_data[0])
    num_class_series =  np.unique(y_data, return_counts=True)
    num_class_possbilities = num_class_series[1] / num_total_case
    
    case_indexes = []
    for case in num_class_series[0]:
        case_indexes.append(np.where(y_data == case)[0])
        
    possbiliteis_vector = []
    for i, case_index in enumerate(case_indexes):
        possbiliteis_vector.append( (np.sum(x_data[case_index], axis=0)+1  ) / (num_class_series[1][i]+2) )
    return num_class_possbilities, possbiliteis_vector

def infsum(data, **args):
    return np.ma.filled(np.ma.masked_array(data,np.isinf(data)).sum(**args), fill_value=np.nan)

In [252]:
from sklearn.cross_validation import KFold
kf = KFold(len(y_data), n_folds=2)

for train_index, test_index in kf:
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    num_class_possbilities, possbiliteis_vector = train_navie_bayes_possiblities(x_train,y_train)

    expected_possbilites = []
 
    for i, case in enumerate(num_class_possbilities): 
        sum_vector = np.log(x_test * possbiliteis_vector[i])
        sum_possbilities = infsum(sum_vector, axis=1)
        expected_possbilites.append(sum_possbilities)
    
    clasfiy_vector = np.argmax(np.array(expected_possbilites), axis=0)
    print(sum(clasfiy_vector == y_data[test_index].ravel())/ len(y_data[test_index].ravel()))


0.662980583773
0.674511316872


In [253]:
from sklearn import linear_model, datasets


for train_index, test_index in kf:
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    logreg = linear_model.LogisticRegression(C=1e5)
    logreg.fit(x_train, y_train.ravel())
    
    print(sum(logreg.predict(x_test) == y_test.ravel()) / y_data.shape[0])

0.37954092458
0.378062110204
